In [3]:
import random

from matplotlib.pyplot import minorticks_on
from tqdm import tqdm
import src.generate_encodings as ge
import src.prediction_models as pm
import src.predictor_optimizer as pop
import src.mutant_discovery as dis
import numpy as np
import torch
import warnings
import os, sys
from sklearn.metrics import root_mean_squared_error

In [4]:
"""Identified esmc-Params"""
params = {
    "rf": {},
    "xgboost": {'max_depth': 8, 'min_child_weight': 3.2593353403912118, 'subsample': 0.8486626985820169, 'colsample_bytree': 0.5808341075955024, 'eta': 0.23818695386489855},
    "lightgbm": {'num_boost_round': 263, 'learning_rate': 0.06834905202765895, 'num_leaves': 246, 'bagging_fraction': 0.7767804359356935, 'max_depth': 17, 'max_bin': 898}
}

n_layers = 36  #esmc_600 has 36 layers
results = []
data_set = "GRB2_HUMAN_Faure_2021"
mapping_file = f"../Data/Embeddings/{data_set}/pairings.csv"
method = "esmc_600m"
cv_folds = 5





In [5]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


class HiddenWarnings():
    def __enter__(self):
        # Save the current filter settings before changing them
        self._previous_filters = warnings.filters[:]
        # Ignore all warnings
        warnings.filterwarnings("ignore")

    def __exit__(self, exc_type, exc_val, exc_tb):
        # Restore the original warning filter settings
        warnings.filters = self._previous_filters

In [10]:
"""iterate through all embedding layers of esmc represenations from the dataset and test them out with different models"""

data_set_name_shortened = data_set.split("_")[0]

with open(f"../Results/esmc_600m_layer_comparison_{data_set_name_shortened}.csv", "w") as f:
    f.write(f"Comparing all layers of esmc_600m for {data_set} Dataset\n")
    f.write("\n")

for i in range(n_layers - 1, 0, -1):

    #load Embeddings
    representations = []
    scores = []
    repr_layer = i
    path_to_embeddings = f"../Data/Embeddings/{data_set}/{method}/{repr_layer}"

    with open(mapping_file) as f:
        data_pairs = [line[:-1] for line in f.readlines()[1:]]

    with tqdm(total=len(data_pairs), desc=f"loading embeddings of layer {i} to cpu...") as pbar:
        for line in data_pairs:
            line = line[:-1].split(',')
            embedding = f"{path_to_embeddings}/{line[0][1:]}"
            representations.append(torch.load(embedding).to(dtype=torch.float32).cpu())
            scores.append(float(line[1]))
            pbar.update(1)

    #create the models and test them out over whole dataset with tuned hyperparams:
    for m_type in ["xgboost", "lightgbm"]:
        print(f"Now comparing layer {i} to {m_type} with tuned hyperparams")
        model = pm.ActivityPredictor(model_type=m_type,
                                     x_arr=representations,
                                     y_arr=scores,
                                     shuffle_data=True,
                                     early_stopping=15,
                                     params=params[m_type],
                                     seed=random.seed())

        with HiddenPrints():
            with HiddenWarnings():
                model.train(k_folds=cv_folds)
        r2 = round(model.get_performance()[0], 3)
        rmse = round(model.get_performance()[1], 3)

        result = [m_type, repr_layer, "tuned_hyperparams", r2, rmse]
        with open(f"../Results/esmc_600m_layer_comparison_{data_set_name_shortened}.csv", "a") as f:
            f.write(f"{result}\n")

    #create the models and test them out over whole dataset with default hyperparams:
    for m_type in ["xgboost", "lightgbm", "rf", "adaboost"]:
        print(f"Now comparing layer {i} to {m_type} with default hyperparams")
        model = pm.ActivityPredictor(model_type=m_type,
                                     x_arr=representations,
                                     y_arr=scores,
                                     shuffle_data=True,
                                     early_stopping=15,
                                     params={},
                                     seed=random.seed())

        with HiddenPrints():
            with HiddenWarnings():
                model.train(k_folds=cv_folds)
        r2 = round(model.get_performance()[0], 3)
        rmse = round(model.get_performance()[1], 3)

        result = [m_type, repr_layer, "default_hyperparams", r2, rmse]
        with open(f"../Results/esmc_600m_layer_comparison_{data_set_name_shortened}.csv", "a") as f:
            f.write(f"{[field for field in result]}\n")

loading embeddings of layer 35 to cpu...: 100%|██████████| 63366/63366 [00:42<00:00, 1483.84it/s]


Now comparing layer 35 to xgboost with tuned hyperparams
Now comparing layer 35 to lightgbm with tuned hyperparams
Now comparing layer 35 to xgboost with default hyperparams
Now comparing layer 35 to lightgbm with default hyperparams
Now comparing layer 35 to rf with default hyperparams
Now comparing layer 35 to adaboost with default hyperparams


loading embeddings of layer 34 to cpu...: 100%|██████████| 63366/63366 [00:40<00:00, 1558.21it/s]


Now comparing layer 34 to xgboost with tuned hyperparams
Now comparing layer 34 to lightgbm with tuned hyperparams
Now comparing layer 34 to xgboost with default hyperparams
Now comparing layer 34 to lightgbm with default hyperparams
Now comparing layer 34 to rf with default hyperparams
Now comparing layer 34 to adaboost with default hyperparams


loading embeddings of layer 33 to cpu...: 100%|██████████| 63366/63366 [00:40<00:00, 1556.24it/s]


...

Now comparing layer 3 to xgboost with tuned hyperparams
Now comparing layer 3 to lightgbm with tuned hyperparams
Now comparing layer 3 to xgboost with default hyperparams
Now comparing layer 3 to lightgbm with default hyperparams
Now comparing layer 3 to rf with default hyperparams
Now comparing layer 3 to adaboost with default hyperparams


loading embeddings of layer 2 to cpu...: 100%|██████████| 63366/63366 [00:41<00:00, 1529.37it/s]


Now comparing layer 2 to xgboost with tuned hyperparams
Now comparing layer 2 to lightgbm with tuned hyperparams
Now comparing layer 2 to xgboost with default hyperparams
Now comparing layer 2 to lightgbm with default hyperparams
Now comparing layer 2 to rf with default hyperparams
Now comparing layer 2 to adaboost with default hyperparams


loading embeddings of layer 1 to cpu...: 100%|██████████| 63366/63366 [00:37<00:00, 1705.60it/s]


Now comparing layer 1 to xgboost with tuned hyperparams
Now comparing layer 1 to lightgbm with tuned hyperparams
Now comparing layer 1 to xgboost with default hyperparams
Now comparing layer 1 to lightgbm with default hyperparams
Now comparing layer 1 to rf with default hyperparams
Now comparing layer 1 to adaboost with default hyperparams


In [11]:
# for saved resultsfile
with open(f"../Results/esmc_600m_layer_comparison_{data_set_name_shortened}.csv", "r") as f:
    lines = f.readlines()[2:]
results = []
for line in lines:
    line = line[:-1].split(",")
    line[1] = int(float(line[1]))
    line[3] = float(line[3])
    line[4] = float(line[4])
    results.append(line)

models_tested = list(set([(result[0], result[2]) for result in results]))
# print(models_tested)
results = sorted(results, key=lambda x: x[1])
print(results)

[['xgboost', 1, ' tuned_hyperparams', 0.741, 0.215], ['lightgbm', 1, ' tuned_hyperparams', 0.755, 0.208], ['xgboost', 1, ' default_hyperparams', 0.73, 0.219], ['lightgbm', 1, ' default_hyperparams', 0.674, 0.239], ['rf', 1, ' default_hyperparams', 0.623, 0.264], ['adaboost', 1, ' default_hyperparams', 0.17, 0.385], ['xgboost', 2, ' tuned_hyperparams', 0.761, 0.207], ['lightgbm', 2, ' tuned_hyperparams', 0.783, 0.196], ['xgboost', 2, ' default_hyperparams', 0.759, 0.206], ['lightgbm', 2, ' default_hyperparams', 0.712, 0.228], ['rf', 2, ' default_hyperparams', 0.639, 0.253], ['adaboost', 2, ' default_hyperparams', 0.27, 0.363], ['xgboost', 3, ' tuned_hyperparams', 0.764, 0.206], ['lightgbm', 3, ' tuned_hyperparams', 0.786, 0.196], ['xgboost', 3, ' default_hyperparams', 0.756, 0.209], ['lightgbm', 3, ' default_hyperparams', 0.709, 0.231], ['rf', 3, ' default_hyperparams', 0.647, 0.251], ['adaboost', 3, ' default_hyperparams', 0.274, 0.359], ['xgboost', 4, ' tuned_hyperparams', 0.761, 0.20

In [12]:
"""Display Results: Modelperformance for every Layer in esmc_600m"""

from plotly.subplots import make_subplots
import plotly.graph_objects as go

results_plot = make_subplots(
    rows=1, cols=1)

colors = {
    ('xgboost', ' default_hyperparams'): "cyan",
    ('xgboost', ' tuned_hyperparams'): "darkcyan",
    ('rf', ' default_hyperparams'): "yellow",
    ('lightgbm', ' default_hyperparams'): "red",
    ('lightgbm', ' tuned_hyperparams'): "darkred",
    ('adaboost', ' default_hyperparams'): "grey"
}

for i, model in enumerate(models_tested):
    x= [result[1] for result in results if result[0] in model and result[2] in model]
    y= [result[3] for result in results if result[0] in model and result[2] in model]

    results_plot.append_trace(
        go.Scatter(name="_".join(model),
                   x=x,
                   y=y,
                   marker=dict(color=colors[model], size=3),
                   mode="lines"), row=1, col=1)

#
results_plot.update_layout(
    title_text=f"Performance of selected tested models for {data_set} Dataset for all sequence representation layers for {method}.",
    title_font=dict(color="black", size=20),
    showlegend=True,
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
    width=1400,
    height=1000,
    legend=dict(font=dict(color="black",
                          size=12)))
#
results_plot.update_yaxes(
    dict(
        title_text="R2 Performance",
        title_font=dict(color="black"),
        range=[0, 1],
        color='black',
        showgrid=True,
        gridcolor='grey',
        griddash="dot",
        dtick=0.1,
        gridwidth=1)
)

results_plot.update_xaxes(
    dict(
        title_text="cycles",
        title_font=dict(color="black"),
        range=[1, 35],
        color='black',
        showgrid=True,
        gridcolor='grey',
        dtick=1,
        griddash="dot",
        gridwidth=1)
)

import plotly.io as pio

pio.renderers.default = "browser"
with HiddenWarnings():
    results_plot.show()

import kaleido
results_plot.write_image(f"ESMC_600m_layer_comparison_{data_set_name_shortened}.jpg")

Gtk-Message: 15:18:03.705: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.
[255482, Main Thread] WARNING: GTK+ module /snap/firefox/6103/gnome-platform/usr/lib/gtk-2.0/modules/libcanberra-gtk-module.so cannot be loaded.
GTK+ 2.x symbols detected. Using GTK+ 2.x and GTK+ 3 in the same process is not supported.: 'glib warning', file /build/firefox/parts/firefox/build/toolkit/xre/nsSigHandlers.cpp:201

(firefox_firefox:255482): Gtk-WARNING **: 15:18:03.755: GTK+ module /snap/firefox/6103/gnome-platform/usr/lib/gtk-2.0/modules/libcanberra-gtk-module.so cannot be loaded.
GTK+ 2.x symbols detected. Using GTK+ 2.x and GTK+ 3 in the same process is not supported.
Gtk-Message: 15:18:03.755: Failed to load module "canberra-gtk-module"
[255482, Main Thread] WARNING: GTK+ module /snap/firefox/6103/gnome-platform/usr/lib/gtk-2.0/modules/libcanberra-gtk-module.so cannot be loaded.
GTK+ 2.x symbols detected. Using GTK+ 2.x and GTK+ 3 in the s